In [ ]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

output = join("output","correcoes","casos_confirmados")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [ ]:
timer.reset()
tb = TbPacientes()
tb.update()
timer.stop()

In [ ]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

In [ ]:
# casos = casos_confirmados.fix_mun_resid_casos('mun_atend')
# obitos = casos_confirmados.fix_mun_resid_obitos()
casos = casos_confirmados.get_casos()
casos = casos.drop(columns=['obito','data_obito','excluir'])
obitos = casos_confirmados.get_obitos()

casost = tb.get()

print(f"{len(casos)};{len(obitos)}")
obitos.loc[~obitos['hash'].isin(casos['hash'])]

In [ ]:
obitos = obitos.rename(columns={'data_do_obito': 'data_obito'})
obitos['obito'] = 'SIM'

casos = pd.merge(left=casos, right=obitos[['obito','data_obito','hash']], on='hash', how='left')
casos.loc[casos['obito'].isnull(), 'obito'] = 'NAO'
# casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'obito'] = 'NAO'
# casos.loc[(casos['obito']=="SIM") & (casos.duplicated('hash')), 'data_obito'] = pd.NaT

print(f"{len(casos)};{len(casos.loc[casos['obito']=='SIM'])}")

In [ ]:
casos['excluir'] = 'NAO'

print(f"Duplicados com a mesma idade: {casos.loc[casos.duplicated('hash', keep=False)].shape}")
print(f"Duplicados com idade menos 1: {casos.loc[casos['hash_less'].isin(casos['hash'])].shape}")
casos.loc[casos['hash_less'].isin(casos['hash']), 'hash'] = casos.loc[casos['hash_less'].isin(casos['hash']), 'hash_less']
print(f"Duplicados com idade mais 1: {casos.loc[casos['hash_more'].isin(casos['hash'])].shape}")
casos.loc[casos['hash_more'].isin(casos['hash']), 'hash'] = casos.loc[casos['hash_more'].isin(casos['hash']), 'hash_more']
print(f"todos os duplicados: {casos.loc[casos.duplicated('hash', keep=False)].shape}")
duplicados = casos.loc[casos.duplicated('hash', keep=False)]
print(f"Valores unicos: {len(duplicados.groupby('hash'))}")

In [ ]:
casost['excluir'] = 'NAO'

print(f"Duplicados com a mesma idade: {casost.loc[casost.duplicated('hash', keep=False)].shape}")
print(f"Duplicados com idade menos 1: {casost.loc[casost['hash_less'].isin(casost['hash'])].shape}")
casost.loc[casost['hash_less'].isin(casost['hash']), 'hash'] = casost.loc[casost['hash_less'].isin(casost['hash']), 'hash_less']
print(f"Duplicados com idade mais 1: {casost.loc[casost['hash_more'].isin(casost['hash'])].shape}")
casost.loc[casost['hash_more'].isin(casost['hash']), 'hash'] = casost.loc[casost['hash_more'].isin(casost['hash']), 'hash_more']
print(f"todos os duplicados: {casost.loc[casost.duplicated('hash', keep=False)].shape}")
duplicadost = casost.loc[casost.duplicated('hash', keep=False)]
print(f"Valores unicos: {len(duplicadost.groupby('hash'))}")

In [ ]:
if len(duplicados.groupby('hash')) > len(duplicadost.groupby('hash')):
    print(len(duplicados.groupby('hash'))-len(duplicadost.groupby('hash')))
    raise Exception(':/')
elif len(duplicados.groupby('hash')) < len(duplicadost.groupby('hash')):
    print(len(duplicadost.groupby('hash'))-len(duplicados.groupby('hash')))
    raise Exception(':/')
else: 
    print(':)')

In [ ]:
def get_better(cc,tb):
    if len(cc) != len(tb):
        raise Exception('Error')

    scores = np.zeros(len(cc))

    for i, ((_, cc_row),(_, tb_row)) in enumerate(zip(cc.iterrows(),tb.iterrows())):

        if tb_row['obito'] == 'SIM':
            scores[i] = 10
        elif tb_row['status'] == 'RECUPERADO':
            scores[i] = 10

    i = np.argmax(scores)
    return cc.iloc[i].name, tb.iloc[i].name

In [ ]:
manter = []
tb_manter = []
for cc_group, tb_group in zip(duplicados.groupby('hash'),duplicadost.groupby('hash')):
    if cc_group[0] != tb_group[0]:
        print(cc_group[0])
        raise Exception(':/')
    
    idx, tb_idx = get_better(cc_group[1],tb_group[1])

    manter.append(idx)
    tb_manter.append(tb_idx)

In [ ]:
duplicados = set(duplicados.index.tolist())
duplicados = duplicados - set(manter)

casos.loc[duplicados,'excluir'] = 'SIM'
casos.loc[duplicados,'hash'] = None
casos.loc[duplicados,'hash_less'] = None
casos.loc[duplicados,'hash_more'] = None

In [ ]:
duplicadost = set(duplicadost.index.tolist())
duplicadost = duplicadost - set(tb_manter)

casost.loc[duplicadost,'excluir'] = 'SIM'
casost.loc[duplicadost,'hash'] = None
casost.loc[duplicadost,'hash_less'] = None
casost.loc[duplicadost,'hash_more'] = None

In [ ]:
print(casos.loc[casos['excluir']=='SIM'].shape)
print(casost.loc[casost['excluir']=='SIM'].shape)
# exit()

In [ ]:
new_casos = casos[['ordem','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is','obito','data_obito','excluir']]
new_obitos = obitos[['ordem','ibge7','nome','sexo','idade','municipio','rs','data_obito']]

In [ ]:
writer = pd.ExcelWriter(join(output,"Casos confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

new_casos.to_excel(writer,sheet_name='Casos confirmados',index=None)
new_obitos.to_excel(writer,sheet_name='Obitos',index=None)

writer.save()
writer.close()

In [18]:
# casost = casost.drop(columns=['hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend'])

casost['ibge_res_pr'] = casost['ibge_res_pr'].fillna('9999999')
casost['ibge_atend_pr'] = casost['ibge_atend_pr'].fillna('9999999')

casost = casost.loc[casost['excluir']=='NAO']
casost = casost.drop(columns=['excluir'])

writer = pd.ExcelWriter(join(output,"novos_tb_pacientes.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casost.to_excel(writer,index=None)

writer.save()
writer.close()

In [ ]:
exit()
# new_casos = casos[['ordem','ibge7','nome','sexo','idade','mun_resid','mun_atend','rs','laboratorio','dt_diag','comunicacao','is']]
for grupo_casos, grupo_obitos in zip(new_casos.groupby('rs'),new_obitos.groupby('rs')):
    writer = pd.ExcelWriter(join(output,f"casos_confirmados_{int(grupo_casos[0])}a_regional.xlsx"),
                            engine='xlsxwriter',
                            datetime_format='dd/mm/yyyy',
                            date_format='dd/mm/yyyy')

    grupo_casos[1].to_excel(writer,sheet_name='Casos confirmados',index=None)
    grupo_obitos[1].to_excel(writer,sheet_name='Obitos',index=None)

    writer.save()
    writer.close()

In [ ]:
writer = pd.ExcelWriter(join(output,f"casos_confirmados_nao_residentes.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

new_casos.loc[new_casos['rs'].isnull()].to_excel(writer,sheet_name='Casos confirmados',index=None)
new_obitos.loc[new_obitos['rs'].isnull()].to_excel(writer,sheet_name='Obitos',index=None)

writer.save()
writer.close()